In [13]:
from openai import OpenAI
import requests 
import json
import os
from dotenv import load_dotenv

In [14]:
load_dotenv()

True

In [15]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [16]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

In [17]:


SYSTEM_PROMPT = """
You are a Lead PM Interviewer. Your task is to vet the user's response against the 
provided Interview Question using these 7 criteria:
1. Structure and Clarity
2. Creativity and Innovation
3. Depth of Analysis
4. Market Understanding
5. Business and Strategy
6. Technical Feasibility
7. Personalization and Decision Making

For each criterion, output ONLY:
- Positive Aspects (1-3 bullets, 1-2 lines each)
- Areas for Improvement (1-3 bullets, 1-2 lines each)
- Actionable Recommendations (1-3 bullets, 1-2 lines each)

Be clinical, demanding, and objective.
"""

question = "How would you design a travel app for solo female travelers?"
user_answer = """irst, aggregate hyperlocal safety signals (crowd density, lighting, reviews from verified women, incident heatmaps) and integrate live check-ins with trusted contacts, so safety becomes a dynamic layer over the city. Second, build a verified community layer where solo female travelers can join micro-groups for specific dates/locations, share itineraries, and rate experiences specifically on safety, not just aesthetics. Finally, compress planning into actionable flows—curated safe stays, transport options with reliability scores, and AI-assisted itinerary building that optimizes for safety, convenience, and local authenticity simultaneously."""

user_content = f"""
### INTERVIEW QUESTION
{question}

---
### CANDIDATE RESPONSE FOR EVALUATION
{user_answer}
"""

In [18]:

response = client.chat.completions.create(
  model="stepfun/step-3.5-flash:free",
  messages=[
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_content}
  ],
  temperature=0.4, # Lower temperature for more consistent, objective vetting
  extra_body={
    "reasoning": {"enabled": True}, # Recommended for deep PM analysis
    "effort": "medium"
  }
)

print(response.choices[0].message.content)

**1. Structure and Clarity**
- **Positive Aspects:** Clearly segments the solution into three logical pillars (safety infrastructure, community, planning efficiency), making the core value proposition easy to grasp.
- **Areas for Improvement:** Lacks a defined implementation roadmap or prioritization (e.g., MVP vs. long-term). The flow from problem to solution is implied but not explicitly structured.
- **Actionable Recommendations:** Begin responses with a concise problem statement (e.g., "Solo female travel is hindered by safety anxiety and planning friction"). Follow with a phased rollout plan: Phase 1 = core safety layer, Phase 2 = community, Phase 3 = AI optimization.

**2. Creativity and Innovation**
- **Positive Aspects:** Introduces novel concepts like "safety heatmaps" and "micro-groups for specific dates/locations," moving beyond generic social features to context-aware, temporary cohorts.
- **Areas for Improvement:** Relies on established patterns (crowdsourcing, AI itinerar

In [19]:
from groq import Groq

In [20]:

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq()
completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ],
    temperature=1,
    max_completion_tokens=8192,
    top_p=1,
    reasoning_effort="medium",
    stream=True,
    stop=None
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

**1. Structure and Clarity**  
*Positive Aspects*  
- Begins with a clear three‑step outline, making the high‑level flow easy to follow.  
- Uses concise phrasing to convey each major component (safety signals, community, planning).  

*Areas for Improvement*  
- Lacks introductory framing that defines the problem scope or user personas.  
- Transitions between steps are abrupt; the logical connection between community and itinerary building is vague.  

*Actionable Recommendations*  
- Open with a brief problem statement and target user description to set context.  
- Use headings or numbered sub‑sections to separate each pillar and show how they interrelate.  

---

**2. Creativity and Innovation**  
*Positive Aspects*  
- Introduces “hyperlocal safety signals” and “incident heatmaps,” which go beyond typical safety ratings.  
- Proposes a “verified community layer” for micro‑groups, adding a social safety net.  

*Areas for Improvement*  
- Concepts remain at a high level; no novel 

In [32]:
INTERVIEW_AGENTS = [
    {
        "code": "PM",
        "role": "Product Manager",
        "prompt": "Evaluate based on: Structure and Clarity, Creativity and Innovation, Depth of Analysis, Market Understanding, Business and Strategy, Technical Feasibility, Personalization and Decision Making."
    },
    {
        "code": "TA",
        "role": "Technical Architect",
        "prompt": "Evaluate based on: Logical Decomposition, System Scalability, Trade-off Analysis, Edge Case Resilience, Clean Code & Patterns, Technical Communication, and Security Mindset."
    },
    {
        "code": "SE",
        "role": "Software Engineer",
        "prompt": "Evaluate based on: Algorithmic Efficiency, Code Readability, Testing & Validation, Problem-Solving Logic, Tool/Framework Mastery, Documentation Quality, and Version Control Best Practices."
    },
    {
        "code": "DS",
        "role": "Data Scientist",
        "prompt": "Evaluate based on: Feature Engineering Logic, Model Selection Reasoning, Handling Bias & Overfitting, Statistical Significance, Deployment Feasibility, Data Intuition, and Evaluation Metrics Selection."
    },
    {
        "code": "HR",
        "role": "Human Resources",
        "prompt": "Evaluate based on: Conflict Resolution, Cultural Alignment, Behavioral Consistency, Communication Style, Growth Mindset, Policy Awareness, and Team Collaboration Potential."
    },
    {
        "code": "GM",
        "role": "Growth Marketer",
        "prompt": "Evaluate based on: Data Literacy, Hypothesis Generation, Channel Mastery, Experimentation Rigor, Creative Intuition, Cross-Functional Ops, and Budget Efficiency."
    },
    {
        "code": "CSL",
        "role": "Customer Success Lead",
        "prompt": "Evaluate based on: De-escalation Skills, Proactive Strategy, Value Realization, Empathy & Active Listening, Relationship Building, Product Expertise, and Expansion Identification."
    },
    {
        "code": "VL",
        "role": "Visionary Leader",
        "prompt": "Evaluate based on: Strategic Agility, Cultural Stewardship, Talent Development, Decision-Making Under Pressure, Influence & Persuasion, Operational Excellence, and Emotional Intelligence (EQ)."
    }
]

# Universal instruction suffix to ensure the model follows your formatting constraints
UNIVERSAL_FORMAT = """
For each of the 7 factors above, provide ONLY:
- Positive Aspects (1-2 bullets, 1-2 lines each)
- Areas for Improvement (1-2 bullets, 1-2 lines each)
- Actionable Recommendations (1-2 bullets, 1-2 lines each)
No intro or outro text. Be clinical, demanding and objective. 
"""